## Definitions:

### Primitive root of unity
Any complex number that yields 1 when raised to some positive integer power n.
1. $ \zeta^n = 1 $
2. $ \zeta^k \neq 1 $ for all integers $ k $ such that $ 0 < k < n $.

$\zeta^n = e^{i(2\pi \frac{k}{n})}$

For 2N-th roots of unity, this means any number $\zeta$ that satisfies $\zeta^{2N}=1$

### Cyclotomic Polynomial
A **cyclotomic polynomial** is a special type of polynomial defined as the unique irreducible polynomial with integer coefficients whose roots are the **primitive roots of unity** $e^{i(2\pi\frac{k}{n})}$, where k runs over the positive integers less than n and coprime to n. The formula is equal to:

$$
\Phi_{n}(x) = \prod_{\substack{1 \leq k \leq n \\ \gcd(k, n) = 1}} \left(x - e^{2i\pi \frac{k}{n}}\right)
$$

**Special Case**: $\Phi_M(X) = X^N + 1$

The specific form $\Phi_M(X) = X^N + 1$ arises when:

- $M = 2N$, i.e., $M$ is an even number and M is a power of 2.
- The roots of $\Phi_{2N}(X)$ are the primitive $2N$-th roots of unity.


#### 2.1. Roots of $X^N + 1$

The equation $X^N + 1 = 0$ has $N$ distinct roots in the complex plane. These roots are the **odd powers** of $\xi$, a primitive $2N$-th root of unity:

$$
\xi = e^{2\pi i / (2N)}, \quad \text{so } \xi^{2k+1} \text{ for } k = 0, 1, \dots, N-1.
$$



#### 2.2. Factorization

The polynomial $X^N + 1$ factors as:

$$
X^N + 1 = \prod_{k=0}^{N-1} \left( X - \xi^{2k+1} \right),
$$

where $\xi^{2k+1}$ are the odd powers of the primitive $2N$-th root of unity $\xi$.



#### 2.3. Minimal Polynomial

When $M = 2N$, the primitive $2N$-th roots of unity are precisely the roots of $X^N + 1$. Hence, the minimal polynomial of the primitive $2N$-th roots of unity is:

$$
\Phi_{2N}(X) = X^N + 1.
$$


### Isomorphism

An isomorphism between two algebraic structures is a bijective homomorphism.

**Addition Preservation**:
$\sigma(m1 + m2) = \sigma(m1) + \sigma(m2)$

**Multiplication Preservation**:
$\sigma(m1 * m2) = \sigma(m1) \circ \sigma(m2)$

- Injectivity: No two distinct polynomials map to the same vector under σ.
- Surjectivity: Every vector in ${C^N}$ (satisfying the necessary properties) is the image of some polynomial under σσ.




## Plaintext Encoding and Decoding


### Encoding

For encoding, we aim to project a complex vector into the subspace:

$$
H = \{(z_j)_{j \in \mathbb{Z}^*_M} : z_j = z_{-j}\}.
$$

This is the $\pi^{-1}$ projection discussed in the original paper, where half of the entries are conjugates of the other half. We know that:

$$
\sigma(\mathcal{R}) \subseteq \mathbb{H} = \{ z \in \mathbb{C}^N : z_j = z_{-j} \}.
$$

The process involves the following steps:

1. **Projection using $\pi^{-1}$:**  
   The scaled vector is then projected using $\pi^{-1}$. This ensures that the resulting vector lies in $\mathbb{H}$.
   


In [45]:
# code inspired from https://colab.research.google.com/drive/1cdue90Fg_EB5cxxTYcv2_8_XxQnpnVWg?usp=sharing#scrollTo=qsDuOVro1H15

from math import sin,cos,pi
import numpy as np
from numpy.polynomial import Polynomial

In [46]:
def pi_inverse(z: np.array) -> np.array:
    """Expands a vector of C^{N/2} by expanding it with its
    complex conjugate."""
    
    z_conjugate = z[::-1]
    z_conjugate = [np.conjugate(x) for x in z_conjugate]
    return np.concatenate([z, z_conjugate])

# TOY EXAMPLE FROM https://eprint.iacr.org/2016/421.pdf section 3.2

M = 8 # parameter from the paper
delta = 64 # scaling factor 
xi = np.exp(2 * np.pi * 1j / M) # Mth root of unity which will be used as a basis for our computation
z = np.array([3 +4j, 2 - 1j]) # vector to encode
pi_z = pi_inverse(z) # pi^-1
pi_z

array([3.+4.j, 2.-1.j, 2.+1.j, 3.-4.j])

2. **Multiplication with $\Delta$:**  
   We start by multiplying the vector with $\Delta$ to scale it appropriately.

In [47]:
scaled_pi_z = delta * pi_z
scaled_pi_z # scaled vector

array([192.+256.j, 128. -64.j, 128. +64.j, 192.-256.j])

3. **Coordinate-Wise Random Rounding:**  
   To project onto $\sigma(\mathcal{R})$, we use a technique called *coordinate-wise random rounding*. This is a method to discretize complex numbers in each coordinate independently while preserving certain statistical properties. For details, refer to [this paper](https://web.eecs.umich.edu/~cpeikert/pubs/toolkit.pdf).

    
   A. **Orthogonal $\mathbb{Z}$-Basis:**
   - The ring $\mathcal{R}$ has an orthogonal $\mathbb{Z}$-basis $\{1, X, \ldots, X^{N-1}\}$.
   - Since $\sigma$ is an isomorphism, $\sigma(\mathcal{R})$ has a corresponding orthogonal $\mathbb{Z}$-basis:
     $$
     \beta = \{b_1, b_2, \ldots, b_N\} = \{\sigma(1), \sigma(X), \ldots, \sigma(X^{N-1})\}.
     $$

In [48]:
def vandermonde(xi: np.complex128, M: int) -> np.array:
    """Computes the Vandermonde matrix from a m-th root of unity."""
    
    N = M //2
    matrix = []
    # We will generate each row of the matrix
    for i in range(N):
        # For each row we select a different root
        root = xi ** (2 * i + 1)
        row = []

        # Then we store its powers
        for j in range(N):
            row.append(root ** j)
        matrix.append(row)
    return matrix

def create_sigma_R_basis(xi, M):
    """Creates the basis (sigma(1), sigma(X), ..., sigma(X** N-1))."""
    return np.array(vandermonde(xi, M)).T
    
sigma_R_basis = create_sigma_R_basis(xi, M)

We can now have a look at the basis $\{\sigma(1), \sigma(X), \ldots, \sigma(X^{N-1})\}$.

In [49]:
sigma_R_basis

array([[ 1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ,
         1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ],
       [ 7.07106781e-01+0.70710678j, -7.07106781e-01+0.70710678j,
        -7.07106781e-01-0.70710678j,  7.07106781e-01-0.70710678j],
       [ 1.79380389e-16+1.j        , -4.67705010e-16-1.j        ,
         7.70954637e-16+1.j        , -1.11479041e-15-1.j        ],
       [-7.07106781e-01+0.70710678j,  7.07106781e-01+0.70710678j,
         7.07106781e-01-0.70710678j, -7.07106781e-01-0.70710678j]])

B. **Projection in the Basis:**
   - For any $z \in \mathbb{H}$, we can write:
     $$
     z = \sum_{i=1}^N z_i b_i,
     $$
     where $z_i$ are the coordinates of $z$ in the basis $\beta$. These coordinates are computed as:
     $$
     z_i = \frac{\langle z, b_i \rangle}{\|b_i\|^2} \in \mathbb{R}.
     $$
    - Here, $\langle x, y \rangle$ is the **Hermitian inner product**:
    $$
    \langle x, y \rangle = \sum_{k=1}^N x_k \overline{y_k},
    $$
    where $\overline{y_k}$ denotes the complex conjugate of $y_k$.

In [50]:
def compute_basis_coordinates(z):
    """Computes the coordinates of a vector with respect to the orthogonal lattice basis."""
    output = np.array([np.real(np.vdot(z, b) / np.vdot(b,b)) for b in sigma_R_basis])
    return output


coordinates = compute_basis_coordinates(scaled_pi_z)

C. **Rounding to Integers:**
   - Since the coordinates $z_i$ are real numbers (not integers), we apply **coordinate-wise random rounding**:
     - Each $z_i$ is rounded to either $\lfloor z_i \rfloor$ or $\lfloor z_i \rfloor + 1$.
     - The probability of rounding to $\lfloor z_i \rfloor + 1$ is proportional to the fractional part of $z_i$.

In [51]:
def round_coordinates(coordinates):
    """Gives the integral rest."""
    coordinates = coordinates - np.floor(coordinates)
    return coordinates

def coordinate_wise_random_rounding(coordinates):
    """Rounds coordinates randonmly."""
    r = round_coordinates(coordinates)
    f = np.array([np.random.choice([c, c-1], 1, p=[1-c, c]) for c in r]).reshape(-1)
    
    rounded_coordinates = coordinates - f
    rounded_coordinates = [int(coeff) for coeff in rounded_coordinates]
    return rounded_coordinates

rounded_coordinates = coordinate_wise_random_rounding(coordinates)
rounded_coordinates

[160, 90, 160, 45]

D. **Reconstruction of $z$:**
   - Once we have the rounded coordinates $\{z_i\}$, we reconstruct $z$ using the basis $\beta$:
     $$
     z = \sum_{i=1}^N z_i b_i.
     $$


In [52]:
y = np.matmul(sigma_R_basis.T, rounded_coordinates)
y

array([191.81980515+255.45941546j, 128.18019485 -64.54058454j,
       128.18019485 +64.54058454j, 191.81980515-255.45941546j])

4. **Applying $\sigma^{-1}$:**  
Finally, we apply $\sigma^{-1}$, as described below. This step outputs an element in $\mathbb{R}$, completing the encoding process.

Find a polynomial $m(X) = \sum_{i=0}^{N-1} \alpha_i X^i \in \mathbb{C}[X]/(X^N + 1)$, given a vector $z \in \mathbb{C}^N$, such that $\sigma(m) = (m(\xi), m(\xi^3), \ldots, m(\xi^{2N-1})) = (z_1, \ldots, z_N)$ where $\xi_M$, the $M$-th root of unity: $\xi_M = e^{2i\pi/M}$. So we get:
$\sum_{j=0}^{N-1} \alpha_j (\xi^{2i-1})^j = z_i, \; i = 1, \ldots, N.$

This can be viewed as a linear equation:

$A\alpha = z$, with $A$ the Vandermonde matrix of the $(\xi^{2i-1})_{i=1,\ldots,N}$, $\alpha$ the vector of the polynomial coefficients, and $z$ the vector we want to encode.

$$
\begin{bmatrix}
1 & \xi_1 & \xi_1^2 & \xi_1^3 \\
1 & \xi_3 & \xi_3^2 & \xi_3^3 \\
1 & \xi_5 & \xi_5^2 & \xi_5^3 \\
1 & \xi_7 & \xi_7^2 & \xi_7^3
\end{bmatrix}
\cdot
\begin{bmatrix}
a_1 \\
a_2 \\
a_3 \\
a_4
\end{bmatrix}
=\begin{bmatrix}
z_1 \\
z_2 \\
z_3 \\
z_4
\end{bmatrix}
$$

where $\xi_1 = \xi_M, \xi_3 = (\xi_M)^3, \xi_5 = (\xi_M)^5, \ldots$.



In [53]:
def sigma_inverse(b: np.array) -> Polynomial:
    """Encodes the vector b in a polynomial using an M-th root of unity."""

    # First we create the Vandermonde matrix
    A = vandermonde(xi, M)

    # Then we solve the system
    coeffs = np.linalg.solve(A, b)

    # Finally we output the polynomial
    p = Polynomial(coeffs)
    
    # We round it afterwards due to numerical imprecision
    coef = np.round(np.real(p.coef)).astype(int)
    return Polynomial(coef)

encoded = sigma_inverse(y)
encoded

Polynomial([160.,  90., 160.,  45.], domain=[-1.,  1.], window=[-1.,  1.], symbol='x')

Therefore, we have that:

$\alpha = A^{-1}z$, and that $\sigma^{-1}(z) = \sum_{i=0}^{N-1} \alpha_i X^i \in \mathbb{C}[X]/(X^N + 1)$. 

### Decoding

For decoding, we work with a cyclotomic polynomial ring $\mathbb{Z}[X]/(X^N + 1)$ and aim to decode it to a vector $z \in \mathbb{C}^N$ such that:
$$
z = \pi \circ \sigma(\Delta^{-1} * m) \in \mathbb{C}^{N/2}.
$$

The decoding process involves the following steps:


1. **Scaling by $1 / \Delta$:**  
   Multiply the polynomial $m$ by the inverse of $\Delta$, i.e., $1 / \Delta$. This rescales the polynomial back to the correct domain.


In [54]:
rescaled_p = encoded / delta

2. **Applying $\sigma$:**  
   Next, apply $\sigma$, which maps the polynomial from $\mathbb{Z}[X]/(X^N + 1)$ to a complex vector in $\mathbb{C}^N$. This is explained in detail below.

To decode a polynomial $m(X)$ into a vector $z$, we evaluate on certain values, which will be the roots of $\Phi(M) = X^N + 1$, where the $N$ roots are $\xi, \xi^3, \ldots, \xi^{2N-1}$. 


In [67]:
def sigmaf(p: Polynomial) -> np.array:
    """Decodes a polynomial by applying it to the M-th roots of unity."""

    outputs = []
    N = M //2

    # We simply apply the polynomial on the roots
    for i in range(N):
        root = xi ** (2 * i + 1)
        output = p(root)
        outputs.append(output)
    return np.array(outputs)

z_vector = sigmaf(rescaled_p)

3. **Projection to $\mathbb{C}^{N/2}$:**  
   The final step is to project the vector onto $\mathbb{C}^{N/2}$. Since the structure ensures that the second half of the array is the conjugate of the first half, this projection simply involves returning the first half of the array.

In [56]:
def pi(z: np.array) -> np.array:
    """Projects a vector of H into C^{N/2}."""
    
    N = M // 4
    return z[:N]

decoded = pi(z_vector)
decoded

array([2.99718446+3.99155337j, 2.00281554-1.00844663j])

If we put everything together we have:

In [69]:
def sigma_R_discretization(z):
    """Projects a vector on the lattice using coordinate wise random rounding."""
    coordinates = compute_basis_coordinates(z)
    
    rounded_coordinates = coordinate_wise_random_rounding(coordinates)
    y = np.matmul(sigma_R_basis.T, rounded_coordinates)
    return y

def decode(p: Polynomial, scale) -> np.array:
        """Decodes a polynomial by removing the scale, 
        evaluating on the roots, and project it on C^(N/2)"""
        rescaled_p = p / scale
        z = sigmaf(rescaled_p)
        pi_z = pi(z)
        return pi_z

def encode(z: np.array, scale) -> Polynomial:
    """Encodes a vector by expanding it first to H,
    scale it, project it on the lattice of sigma(R), and performs
    sigma inverse.
    """
    pi_z = pi_inverse(z)
    scaled_pi_z = scale * pi_z
    rounded_scale_pi_zi = sigma_R_discretization(scaled_pi_z)
    p = sigma_inverse(rounded_scale_pi_zi)
    
    # We round it afterwards due to numerical imprecision
    coef = np.round(np.real(p.coef)).astype(int)
    p = Polynomial(coef)
    return p


# Another example with larger Delta

delta = 2**16
encoded_pol = encode(z, delta)
decoded = decode(encoded_pol, delta)

print(f"Original vector: ", z)
print(f"Encoded polynoial: ", encoded_pol) 
print(f"Vector after encoding and decoding: ", decoded)


Original vector:  [3.+4.j 2.-1.j]
Encoded polynoial:  163840.0 + 92682.0·x + 163840.0·x² + 46341.0·x³
Vector after encoding and decoding:  [3.00000054+4.00000162j 1.99999946-0.99999838j]


We will use this encoded poynomial $163840.0 + 92682.0·x + 163840.0·x² + 46341.0·x³$ as an example for encryption and decryption later.

In [58]:
# code from https://github.com/AI-Tech-Research-Lab/Introduction-to-BFV-HE-ML/blob/main/BFV_theory/BFV_theory.ipynb

def Poly(coeffs):
    """
    Helper function to build polynomials, passing a dictionary of coefficients.
    For example, passing {0: 1, 1: -2, 2: 2} returns the polynomial
    2X**2 - 2X + 1
    """
    max_power = max(coeffs.keys())
    _coeffs = np.zeros(max_power + 1)
     
    for i, c in coeffs.items():
        _coeffs[i] = c
        
    return Polynomial(_coeffs)

def pr(p):
    """ 
    Helper function to pretty-print the polynomials, with human order
    of powers, removed trailing .0, etc.
    """
    coefs = p.coef
    res = ""
    
    powers = range(len(coefs)-1, -1, -1)
    for power, coeff in zip(powers, reversed(coefs)):
        if coeff == 0:
            continue
        
        if int(coeff) == coeff:
            coeff = int(coeff)
                  
        sign = "- " if coeff < 0 else "+ "
        
        if power == 0:
            value = abs(coeff)
        elif abs(coeff) != 1:
            value = abs(coeff)
        else:
            value = ""

        power_sign = {0: "", 1: "X"}
        def_power_sign = f"X**{power}"
        
        res += f" {sign}{value}{power_sign.get(power, def_power_sign)}"
        
    if res[1] == "+":
        return res[3:]
    if res[1] == "-":
        return res[1:]

def mod_on_coefficients(polynomial, modulo):
    """
    Apply the modulo on the coefficients of a polynomial.
    """
    coefs = polynomial.coef
    mod_coefs = []
    for c in coefs:
        mod_coefs.append(c % modulo)
        
    return Polynomial(mod_coefs)

def round_on_nearest_integer(polynomial):
    """
    Round the coefficients of a polynomial to the
    nearest integer.
    """
    coefs = polynomial.coef
    round_coefs = []
    for c in coefs:
        round_coefs.append(round(c))
    
    return Polynomial(round_coefs)


def reduce_polynomial(polynomial, cyclotomic_polynomial, q):
    """
    Reduce a polynomial modulo a cyclotomic polynomial and a modulus.
    """
    remainder = divmod(polynomial, polynomial_modulus)[1] # polynomial modulo cyclotomic polynomial -> get the remainder
    return mod_on_coefficients(remainder, q) # perfrom mod operation on coefficients


## Ciphertext Encryption and Decryption

The **Cheon-Kim-Kim-Song** (CKKS) scheme is an homomorphic encryption (HE) scheme which allows the computation of some functions directly on encrypted data.

In particular, it allows the computation of additions and multiplication:

- between ciphertexts and ciphertexts;
- between ciphertexts and plaintexts.

The result of the operations, once decrypted, is the same as if it were applied on the corresponding plaintexts.

### Typical encryption:
$6.5 + 3.5 \neq 18.3$

E ⬇️ &nbsp; E ⬇️ &nbsp; D ⬆️

$18.5 + 11.3 = 19.8$

### Homomorphic encryption:
$6.5 + 3.5 \neq 10.0$

E ⬇️ &nbsp; E ⬇️ &nbsp; D ⬆️

$11.7 + 3.6 = 15.3$


The schemes like BFV or also CKKS are based on a hard computation problem called Ring Learning With Errors.

### Coefficients

First, the coefficients of the polynomials in RLWE are whole numbers, and they are always taken **modulo** some fixed number $q$. This means that each coefficient is the **remainder** when divided by $q$, effectively keeping them within a limited range.

#### Example:
Let’s take the modulus $q = 12$. You can think of this like a **clock**, where each hour corresponds to an element in the range $\{0, 1, 2, \dots, 11\}$. On a clock, if you add $10 + 5$, the result isn’t $15$—instead, it wraps around to $3$.

Similarly, in modular arithmetic, coefficients work the same way. For example:

$$
10 + 5 \mod 12 = 3
$$

---

#### Ring of Polynomials
Now assume we are working with polynomials instead of just numbers. A polynomial like:

$$
f(x) = 10x + 5
$$

is in the "ring of polynomials," where each coefficient $10$ and $5$ is taken **modulo $q$**. So, if $q = 12$, the polynomial remains valid and computations wrap around, just like on the clock.

### Polynomial Modulus

The second important aspect is that the **polynomials themselves are also reduced modulo a special polynomial**. This special polynomial is called the **polynomial modulus**. Every polynomial used in the scheme is divided by this polynomial modulus, and only the remainder is kept.

In homomorphic encryption (HE) schemes, the polynomial modulus is commonly chosen in the form:

$$
f(x) = x^N + 1
$$

where $N$ is a power of $2$. For example, if we take $N = 8$, the polynomial modulus becomes:

$$
f(x) = x^8 + 1
$$

---

### Key Characteristics of Polynomials in the Scheme

The polynomials used in the scheme have two important properties:

1. **Coefficients are reduced modulo $q$**:
   Each coefficient in the polynomial is reduced modulo $q$, creating a ring structure where coefficients belong to the range $\{0, 1, 2, \dots, q-1\}$.

2. **Maximum degree is $N-1$**:
   The polynomials are reduced modulo the polynomial $f(x) = x^N + 1$, meaning their degree can be at most $N-1$.

---

If $q = 23$ and $N = 8$, then the polynomials in the scheme are of the form:

$$
p(x) = a_0 + a_1x + a_2x^2 + \dots + a_7x^7
$$

where each coefficient $a_i$ satisfies $0 \leq a_i < 23$. This means there are 8 coefficients, each in the range from $0$ to $22$.

### Practical Example
Let's take 2 polynomials $a = 3x^{4}$ and $b = 4x^{5}$ with $N=8$ -> polynomial modulous $x^{8} + 1$ and $q=7$. When we multiple $a*b$, we get $12x^9$.

After each operation, we have to perform the division with the polynomial modulus and keep the remainder.

$$
12x^{9} \mod (x^8 + 1) = -12x
$$

Also, the result's coefficients have to be taken $\mod q$.

$[-12x]_q = 2x$

In [59]:
a = Poly({4: 3})
b = Poly({5: 4})
q = 7
polynomial_modulus = Poly({0: 1, 8: 1})

prod = a * b

quo, rem = divmod(prod, polynomial_modulus)
final_result = mod_on_coefficients(rem, q)

print(f"A: {pr(a)}")
print(f"B: {pr(b)}")
print(f"Product of A and B: {pr(prod)}")
print(f"Polynomial modulus: {pr(polynomial_modulus)}")
print(f"----------------------")
print(f"Remainder of (A*B) / polynomial modulus: {pr(rem)}")
print(f"Apply also mod k: {pr(final_result)}")

A: 3X**4
B: 4X**5
Product of A and B: 12X**9
Polynomial modulus: X**8 + 1
----------------------
Remainder of (A*B) / polynomial modulus: - 12X
Apply also mod k: 2X


### CKKS Encryption and Decryption

In CKKS, encryption takes a plaintext (typically a vector of real or complex numbers) and converts it into a ciphertext using a **public key** derived from a **private key**. The decryption process, which recovers the plaintext, is feasible only if you know the private key.

---

#### Ciphertext
The encryption of a plaintext produces a ciphertext, which is represented by **two or more polynomials** from the same ring. However, the ciphertext uses:
- The same **polynomial modulus** $f(x) = x^N + 1$.
- A much larger modulus $q$, known as the **ciphertext coefficient modulus**, to allow space for noise during homomorphic computations.


#### Example with Small Parameters
To simplify, let’s consider smaller, insecure parameters for an example:
- $N = 4$, so $f(x) = x^4 + 1$..
- $q = ^{60} - 1 $ (ciphertext modulus).

##### Example Plaintext:
A plaintext polynomial might be:

$$
p(x) = 163840 + 92682x + 163840x^2 + 46341^3
$$


##### Encryption:
Encryption transforms $p(x)$ into a ciphertext $(c_0(x), c_1(x))$, where both $c_0(x)$ and $c_1(x)$ are polynomials with coefficients reduced modulo $q = 2^{60} - 1 $.


In [60]:
q = 2**60 - 1 
N = 4
m = Poly({0: 163840, 1: 92682, 2: 163840, 3:46341})
polynomial_modulus = Poly({0: 1, N: 1})


print(f"Ciphertext coefficient modulo: {q}")
print(f"Polynomial modulus: {pr(polynomial_modulus)}")
print(f"Plaintext polynomial: {pr(m)}")

Ciphertext coefficient modulo: 1152921504606846975
Polynomial modulus: X**4 + 1
Plaintext polynomial: 46341X**3 + 163840X**2 + 92682X + 163840


#### Generation of Private Key $s$

In CKKS (and similar schemes), the private key $s$ is a randomly generated polynomial with certain constraints. Here’s a brief explanation:

1. **Parameter $h$**:
   - The parameter $h$ determines the number of non-zero coefficients in the secret key $s$.

2. **Structure of $s$**:
   - $s(x)$ is a polynomial in the ring $R = \mathbb{Z}[x]/(f(x))$, where $f(x) = x^N + 1$.
   - The coefficients of $s(x)$ are either $-1$, $0$, or $1$.

3. **Generation Process**:
   - Randomly select $h$ positions in the polynomial $s(x)$ to be non-zero.
   - Assign these positions a value of $1$ or $-1$ (randomly chosen).
   - The remaining coefficients are set to $0$.

In [61]:
import random


def secret_key_pol(N, h):
    """
    Generate the secret key.
    N = degree of polynomial modulous
    h = number of non-zero coefficients in the polynomial generated
    """
    s = {}
    non_zero_indices = random.sample(range(N), h)
    
    # Assign ±1 randomly to the selected positions
    for idx in non_zero_indices:
        s[idx] = random.choice([-1, 1])
    
    return Poly(s)

h = 2
s = secret_key_pol(N, h) # secret key
print(f"Secret key: {pr(s)}")

Secret key: - X**3 - 1


#### Generation of Public Key $(a, b)$

In CKKS (and similar schemes), the public key is derived from the private key $s$ and includes two polynomials $(a, b)$. Here’s a brief explanation:

1. **Structure of the Public Key**:
   - The public key $(a, b)$ consists of two polynomials in the ring $R = \mathbb{Z}[x]/(f(x))$, where $f(x) = x^N + 1$.
   - The coefficients of these polynomials are reduced modulo a large number $q$ (ciphertext modulus).

2. **Generation Process**:
   - Randomly generate a polynomial $a(x)$ with coefficients modulo $q$.
   - Compute $b(x)$ as:
     $$
     b(x) = -a(x) \cdot s(x) + e(x) \pmod{q}
     $$
     where:
     - $s(x)$ is the private key.
     - $e(x)$ is a small error (noise) polynomial, with coefficients sampled from a discrete Gaussian or uniform distribution to ensure security.

3. **Result**:
   - The public key is $(a(x), b(x))$, where:
     - $a(x)$ is the random polynomial.
     - $b(x)$ is derived using the private key and noise.


In [62]:
def create_error_pol(N, sigma, mu):
    """
    Generate an error polynomial.
    """
    e = np.random.normal(mu, sigma, N)
    e_map = {}
    for i in range(len(e)):
        e_map[i] = round(e[i])
    
    return Poly(e_map)

def sample_pol(N, modulous):
    """
    Sample a polynomial.
    """
    coeffs = {}
    for i in range(N):
        c = np.random.randint(0, modulous)
        coeffs[i] = c
    return Poly(coeffs)


mu, sigma = 0, 3.2 # suggested values in the original paper
a = sample_pol(N, q)
e = create_error_pol(N, sigma, mu)

b = (-a*s)+e
b = reduce_polynomial(b, polynomial_modulus, q)


# public key
pk = (b, a)

print(f"a: {pr(a)}\n")
print(f"e: {pr(e)}\n")
print(f"Public key: {pr(pk[0])}, {pr(pk[1])}\n")

a: 270113418135646016X**3 + 819174886390993664X**2 + 51583833514663072X + 931735272110775424

e: - 5X**2 - 5

Public key: 48927185639574528X**3 + 549061468255347648X**2 + 385330451730516352X + 880151438596112384, 270113418135646016X**3 + 819174886390993664X**2 + 51583833514663072X + 931735272110775424



To perform the encryption we need three "small" polynomials:

- Two error polynomials ("small error" polynomials), extracted from a discrete Gaussian distribution (similarly to the one used in the public key);
- A "small" polynomial, $v$ which has coefficients drawn from $(-1, 0, 1)$, similar to $s$.

In [63]:
e1 = create_error_pol(N, sigma, mu)
e2 = create_error_pol(N, sigma, mu)

coeffs = {}
for i in range(0, N):
    coeffs[i] = np.random.randint(-1, 2)    
v = Poly(coeffs)

print(f"e_1: {pr(e1)}\n")
print(f"e_2: {pr(e2)}\n")
print(f"v: {pr(v)}\n")

e_1: 7X**3 + 6X**2 + 2X - 5

e_2: - 4X**2 - 3X - 2

v: X**3 + X**2 - X



### Encryption in CKKS

In CKKS, the ciphertext is represented by two polynomials:

$$
\text{ct} = \left( \left[ pk[0] \cdot v + e_1 + m \right]_{\Phi_N, q}, \, \left[ pk[1] \cdot v + e_2 \right]_{\Phi_N, q} \right)
$$

Where:
- $m$: The plaintext polynomial
- $v$: A random polynomial for security.
- $e_1, e_2$: Small noise polynomials.
- $q$: Ciphertext modulus.
- $\Phi_N = x^N + 1$: The polynomial modulus.

In [64]:
from math import floor

ct0 = pk[0] * v + e1 + m
ct0 = reduce_polynomial(ct0, polynomial_modulus, q)

ct1 = pk[1] * v + e2
ct1 = reduce_polynomial(ct1, polynomial_modulus, q)

ciphertext = (ct0, ct1)
print(f"Ciphertext: {pr(ct0)}, {pr(ct1)}")

Ciphertext: 716420422071327488X**3 + 445893801226185344X**2 + 827702916722752000X + 267456770260721408, 164144219234444800X**3 + 610038020460466432X**2 + 284819432576279040X + 552276202836836352


### Decryption in CKKS

### Decryption in CKKS

The decryption process in CKKS is relatively straightforward. The ciphertext is represented as two polynomials $(c_0, c_1)$, and decryption uses the private key $s$.

Multiply the second term of the ciphertext $c_1$ with the private key $s$, and sum it with the first term $c_0$:
   $$
   \text{Dec} = c_0 + c_1 \cdot s \pmod{\Phi_N, q}
   $$

   Substituting the ciphertext structure, we get:
   $$
   \text{Dec} = \left[\Delta \cdot m - e_v - e_1 + e_2\cdot s\right]_{\Phi_N, q}
   $$

Inside this polynomial we have the scaled message summed to some noise. If the noise is not too big, we can recover the message.

To do that, we just try to make the modulo with the polynomial modulus, than to apply
to the coefficients of the resulting polynomial.

In [65]:
plaintext = ciphertext[1] * s + ciphertext[0]
plaintext = reduce_polynomial(plaintext, polynomial_modulus, q)

print(f"Plain starting message: {pr(m)}")
print(f"Final decryption result: {pr(plaintext)}")

Plain starting message: 46341X**3 + 163840X**2 + 92682X + 163840
Final decryption result: 46336X**3 + 163712X**2 + 92416X + 164096


Then if we try to decode the decrypted result, we get:

In [70]:
delta = 2**16
decoded_vector = decode(plaintext, delta)

print(f"Original vector: ", z)
print(f"Decoded vector: {decoded_vector}")

Original vector:  [3.+4.j 2.-1.j]
Decoded vector: [3.00109071+3.99512451j 2.00672179-1.00096924j]


Following the decoding process, we obtain a high-quality approximation of the original input vector.

The steps achieved thus far can be summarized as follows:

**Input Vector → Encoding → Encryption → Decryption → Decoding → Approximation of Input Vector**

### Homomorphic Operations

The core of homomorphic encryption are the operations that can be done on encrypted data, that includes addition on ciphertext and plaintext, ciphertext and ciphertext, as well as multiplication of ciphertext and plaintext, as well as ciphertext and ciphertext. Another example that will be showin is squaring a ciphertext.

A homomorphic operation has an effect on the size of plaintext and the growth of message and noise. Each ciphertext will be tagged with bounds of a message and an error in order to dynamically manage their magnitudes. Hence, a full ciphertext will be of the form (c,l,ν,B) for a ciphertext vector c ∈ R, a level 0 ≤ l ≤ L, an upper bound ν ∈ R of message and an upper bound B ∈ R of noise

## Addition

Addition in CKKS is rather straightforward, we have the following ciphertexts:
$$ c_1 = (c_{10}, c_{11})$$
$$ c_2 = (c_{20}, c_{21})$$

$$ c_{add} = c_1 + c_2 = (c_{10} + c_{20}, c_{11} + c_{21})$$

With addition, the resulting ciphertext contains both the original message terms and an accumulated error term. Since the noise terms are typically miniscule compared to the scaling factor, the accumulated error remains manageable.

In [89]:
def add(c_1, c_2):
    c_add_0 = [x + y for x, y in zip(c_1[0], c_2[0])]
    c_add_1 = [x + y for x, y in zip(c_1[1], c_2[1])]

    c_add_0 = Poly({i: coeff for i, coeff in enumerate(c_add_0)})
    c_add_1 = Poly({i: coeff for i, coeff in enumerate(c_add_1)})
    return (c_add_0, c_add_1)

m1 = Poly({0: 163840, 1: 92682, 2: 123821, 3: 46341})
m2 = Poly({0: 204800, 1: 103456, 2: 294721, 3: 57321})
sum_message = m1+m2

ct10 = pk[0] * v + e1 + m1
ct10 = reduce_polynomial(ct10, polynomial_modulus, q)

ct11 = pk[1] * v + e2
ct11 = reduce_polynomial(ct11, polynomial_modulus, q)

c1 = (ct10, ct11)

ct20 = pk[0] * v + e1 + m2
ct20 = reduce_polynomial(ct20, polynomial_modulus, q)

ct21 = pk[1] * v + e2
ct21 = reduce_polynomial(ct21, polynomial_modulus, q)

c2 = (ct20, ct21)

(c_add0, c_add1) = add(c1, c2)
ciphertext = (c_add0, c_add1)

plaintext = ciphertext[1] * s + ciphertext[0]
plaintext = reduce_polynomial(plaintext, polynomial_modulus, q)

print(f"Message 1: {pr(m1)}")
print(f"Message cipher text 1: {pr(ct10)}, {pr(ct11)}")
print(f"Message 2: {pr(m2)}")
print(f"Message cipher text 2: {pr(ct20)}, {pr(ct21)}")

print(f"Added ciphertext: {pr(c_add0)}, {pr(c_add1)}")

print(f"Message 1 + Message 2: {pr(sum_message)}")
print(f"Final decryption result: {pr(plaintext)}")


Message 1: 46341X**3 + 123821X**2 + 92682X + 163840
Message cipher text 1: 716420422071327488X**3 + 445893801226145344X**2 + 827702916722752000X + 267456770260721408, 164144219234444800X**3 + 610038020460466432X**2 + 284819432576279040X + 552276202836836352
Message 2: 57321X**3 + 294721X**2 + 103456X + 204800
Message cipher text 2: 716420422071338496X**3 + 445893801226316224X**2 + 827702916722762752X + 267456770260762368, 164144219234444800X**3 + 610038020460466432X**2 + 284819432576279040X + 552276202836836352
Added ciphertext: 1432840844142665984X**3 + 891787602452461568X**2 + 1655405833445514752X + 534913540521483776, 328288438468889600X**3 + 1220076040920932864X**2 + 569638865152558080X + 1104552405673672704
Message 1 + Message 2: 103662X**3 + 418542X**2 + 196138X + 368640
Final decryption result: 103680X**3 + 418304X**2 + 195584X + 369152


## Multiplication

In [86]:
def multiply(c_1, c_2):
    d_0 = [x * y for x, y in zip(c_1[0], c_2[0])]
    d_1 = [x * y for x, y in zip(c_1[0], c_2[1])] + [x * y for x, y in zip(c_1[1], c_2[0])]
    d_2 = [x * y for x, y in zip(c_1[1], c_2[1])]

    d_0 = Poly({i: coeff for i, coeff in enumerate(d_0)})
    d_1 = Poly({i: coeff for i, coeff in enumerate(d_1)})
    d_2 = Poly({i: coeff for i, coeff in enumerate(d_2)})
    return (d_0, d_1, d_2)

def relin(d_0, d_1, d_2, evk):

    ciphertext = (c_0, c_1)
    return (ciphertext)

def rescale(ciphertext, q):
    return ciphertext

m1 = Poly({0: 163840, 1: 92682, 2: 123821, 3: 46341})
m2 = Poly({0: 204800, 1: 103456, 2: 294721, 3: 57321})
mul_message = m1*m2

ct10 = pk[0] * v + e1 + m1
ct10 = reduce_polynomial(ct10, polynomial_modulus, q)

ct11 = pk[1] * v + e2
ct11 = reduce_polynomial(ct11, polynomial_modulus, q)

c1 = (ct10, ct11)

ct20 = pk[0] * v + e1 + m2
ct20 = reduce_polynomial(ct20, polynomial_modulus, q)

ct21 = pk[1] * v + e2
ct21 = reduce_polynomial(ct21, polynomial_modulus, q)

c2 = (ct20, ct21)

evk = 0
(d_0, d_1, d_2) = multiply(c1, c2)
(c_mul0, c_mul1) = rescale(relin(d_0, d_1, d_2, evk), q)
ciphertext = (c_mul0, c_mul1)

plaintext = ciphertext[1] * s + ciphertext[0]
plaintext = reduce_polynomial(plaintext, polynomial_modulus, q)

print(f"Message 1: {pr(m1)}")
print(f"Message cipher text 1: {pr(ct10)}, {pr(ct11)}")
print(f"Message 2: {pr(m2)}")
print(f"Message cipher text 2: {pr(ct20)}, {pr(ct21)}")

print(f"Multiplied ciphertext: {pr(c_mul0)}, {pr(c_mul1)}")

print(f"Message 1 * Message 2: {pr(mul_message)}")
print(f"Final decryption result: {pr(plaintext)}")


Message 1: 46341X**3 + 123821X**2 + 92682X + 163840
Message cipher text 1: 716420422071327488X**3 + 445893801226145344X**2 + 827702916722752000X + 267456770260721408, 164144219234444800X**3 + 610038020460466432X**2 + 284819432576279040X + 552276202836836352
Message 2: 57321X**3 + 294721X**2 + 103456X + 204800
Message cipher text 2: 716420422071338496X**3 + 445893801226316224X**2 + 827702916722762752X + 267456770260762368, 164144219234444800X**3 + 610038020460466432X**2 + 284819432576279040X + 552276202836836352
Multiplied ciphertext: 1432840844142665984X**3 + 891787602452461568X**2 + 1655405833445514752X + 534913540521483776, 328288438468889600X**3 + 1220076040920932864X**2 + 569638865152558080X + 1104552405673672704
Message 1 * Message 2: 2656312461X**6 + 20755209402X**5 + 46599528359X**4 + 59007466538X**3 + 83234138432X**2 + 35931504640X + 33554432000
Final decryption result: 103680X**3 + 418304X**2 + 195584X + 369152
